In [2]:
import pandas as pd
import time
import pickle

In [3]:
phrases = []
with open('onegin_data.pkl', 'rb') as f:
    phrases = pickle.load(f)

In [4]:
phrases[:10]

['He мысля гордый свет забавить,',
 'Вниманье дружбы возлюбя,',
 'Хотел бы я тебе представить',
 'Залог достойнее тебя,',
 'Достойнее души прекрасной,',
 'Святой исполненной мечты,',
 'Поэзии живой и ясной,',
 'Высоких дум и простоты;',
 'Но так и быть – рукой пристрастной',
 'Прими собранье пёстрых глав,']

In [5]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]

In [6]:
CHARS = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя ')

In [7]:
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]

In [8]:
(INDEX_TO_CHAR)

['none',
 'б',
 'а',
 'г',
 'о',
 ' ',
 'й',
 'и',
 'ч',
 'н',
 'м',
 'ж',
 'в',
 'щ',
 'т',
 'е',
 'д',
 'ё',
 'у',
 'ы',
 'к',
 'р',
 'ш',
 'ц',
 'с',
 'з',
 'я',
 'ф',
 'х',
 'ъ',
 'э',
 'ю',
 'л',
 'п',
 'ь']

In [9]:
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [10]:
CHAR_TO_INDEX

{'none': 0,
 'б': 1,
 'а': 2,
 'г': 3,
 'о': 4,
 ' ': 5,
 'й': 6,
 'и': 7,
 'ч': 8,
 'н': 9,
 'м': 10,
 'ж': 11,
 'в': 12,
 'щ': 13,
 'т': 14,
 'е': 15,
 'д': 16,
 'ё': 17,
 'у': 18,
 'ы': 19,
 'к': 20,
 'р': 21,
 'ш': 22,
 'ц': 23,
 'с': 24,
 'з': 25,
 'я': 26,
 'ф': 27,
 'х': 28,
 'ъ': 29,
 'э': 30,
 'ю': 31,
 'л': 32,
 'п': 33,
 'ь': 34}

In [11]:
import torch

In [12]:
MAX_LEN = 30

In [13]:
X = torch.zeros((len(text), MAX_LEN), dtype=int)

In [14]:
for i in range(len(text)):
    for j, w in enumerate(text[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [15]:
X[0:1]

tensor([[ 0,  0,  5, 10, 19, 24, 32, 26,  5,  3,  4, 21, 16, 19,  6,  5, 24, 12,
         15, 14,  5, 25,  2,  1,  2, 12,  7, 14, 34,  0]])

In [16]:
import random 
def generate_sentence(model):
    j = random.randint(1,len(INDEX_TO_CHAR)-1)
    sentence = [INDEX_TO_CHAR[j]]
    state = None
    for i in range(MAX_LEN):
        X = torch.Tensor([[CHAR_TO_INDEX[sentence[i]]]]).type(torch.long)
        if i == 0:
            result, state = model.forward(X)
        else:
            result, state = model.forward_state(X, state)
        prediction = result[0, -1, :]
        index_of_prediction = prediction.argmax()
        if i >= len(sentence) - 1:
            if index_of_prediction == 0:
                break
        sentence.append(INDEX_TO_CHAR[index_of_prediction])
    print(''.join(sentence))

In [0]:
class Network(torch.nn.Module):

    def __init__(self):
        super(Network, self).__init__()
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 35)
        self.gru = torch.nn.RNN(35, 128, batch_first=True)
        self.hidden2tag = torch.nn.Linear(128, len(INDEX_TO_CHAR))

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [0]:
model = Network()

In [69]:
generate_sentence(model)

мой яёюазгад кдчуъжпшюлжташсйюющра


In [90]:
for ep in range(1000):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers, _ = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    generate_sentence(model1)


Epoch 0. Time: 0.315, Train loss: 0.866
угизы
Epoch 1. Time: 0.334, Train loss: 0.858
жате 
Epoch 2. Time: 0.319, Train loss: 0.856
хотказы в теместе иных 
Epoch 3. Time: 0.286, Train loss: 0.866
хотказы в теместе иных 
Epoch 4. Time: 0.293, Train loss: 0.909
хоткаты славает 
Epoch 5. Time: 0.301, Train loss: 0.926
жате 
Epoch 6. Time: 0.298, Train loss: 0.981
ца 
Epoch 7. Time: 0.289, Train loss: 0.934
хотка не вробил подель но любил
Epoch 8. Time: 0.283, Train loss: 0.903
и мней
Epoch 9. Time: 0.284, Train loss: 0.885
ю 
Epoch 10. Time: 0.291, Train loss: 0.900
говорденский тони ховетит
Epoch 11. Time: 0.286, Train loss: 0.876
дольн их преждечальный свой
Epoch 12. Time: 0.283, Train loss: 0.868
шей рань пором
Epoch 13. Time: 0.275, Train loss: 0.871
говордейс месе разгоре
Epoch 14. Time: 0.286, Train loss: 0.863
шей рань пором
Epoch 15. Time: 0.284, Train loss: 0.859
аставый отарила ной нег
Epoch 16. Time: 0.283, Train loss: 0.859
лыгласкоты
Epoch 17. Time: 0.292, Train loss: 0.867
о

In [0]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.01)

In [16]:
class Network1(torch.nn.Module):

    def __init__(self):
        super(Network1, self).__init__()
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 35)
        self.gru = torch.nn.RNN(35, 256, batch_first=True)
        self.hidden2tag = torch.nn.Linear(256, len(INDEX_TO_CHAR))

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 256))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 256))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [17]:
model1 = Network1()

In [37]:
generate_sentence(model1)

юбый убо лувый


In [23]:
criterion1 = torch.nn.CrossEntropyLoss()
optimizer1 = torch.optim.SGD(model1.parameters(), lr=.01)

In [38]:
for ep in range(1000):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer1.zero_grad()
        answers, _ = model1.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion1(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer1.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    generate_sentence(model1)


Epoch 0. Time: 0.572, Train loss: 0.370
юбый убо лувый
Epoch 1. Time: 0.600, Train loss: 0.369
ить он
Epoch 2. Time: 0.595, Train loss: 0.369
зывоеться
Epoch 3. Time: 0.583, Train loss: 0.369
ых любил
Epoch 4. Time: 0.596, Train loss: 0.369
чивыт
Epoch 5. Time: 0.601, Train loss: 0.369
рыгки милудь и пол
Epoch 6. Time: 0.605, Train loss: 0.369
ьбо жино невеста праздат
Epoch 7. Time: 0.588, Train loss: 0.369
ьбо жино невеста праздат
Epoch 8. Time: 0.584, Train loss: 0.369
эть по другло не
Epoch 9. Time: 0.577, Train loss: 0.369
none после ей наедине
Epoch 10. Time: 0.635, Train loss: 0.369
юбый убо лувый
Epoch 11. Time: 0.570, Train loss: 0.369
рыгки милудь и пол
Epoch 12. Time: 0.608, Train loss: 0.369
носкушей не ветрей медей жетник
Epoch 13. Time: 0.580, Train loss: 0.369
фомо женный рода
Epoch 14. Time: 0.603, Train loss: 0.369
ёт и тамен поветих
Epoch 15. Time: 0.595, Train loss: 0.369
йсёт мож толкно
Epoch 16. Time: 0.602, Train loss: 0.369
носкушей не ветрей медей жетник
Epoch 17

Epoch 138. Time: 0.620, Train loss: 0.365
тойник грустный уваихволо
Epoch 139. Time: 0.583, Train loss: 0.365
йсёт ро стал 
Epoch 140. Time: 0.604, Train loss: 0.365
носкушей не ветрей меский влеон
Epoch 141. Time: 0.553, Train loss: 0.365
стойн
Epoch 142. Time: 0.569, Train loss: 0.364
говораньичиго издлюна
Epoch 143. Time: 0.556, Train loss: 0.364
щей пон пеленый и ножато
Epoch 144. Time: 0.573, Train loss: 0.364
 былог сладков востатья одна по
Epoch 145. Time: 0.576, Train loss: 0.364
ых любил
Epoch 146. Time: 0.582, Train loss: 0.364
ать пртокинет
Epoch 147. Time: 0.657, Train loss: 0.364
вожатн поравляливы злакой
Epoch 148. Time: 0.591, Train loss: 0.364
ёт и тамен порувства
Epoch 149. Time: 0.566, Train loss: 0.364
ых любил
Epoch 150. Time: 0.581, Train loss: 0.364
эть по другло не
Epoch 151. Time: 0.562, Train loss: 0.364
стойн
Epoch 152. Time: 0.571, Train loss: 0.364
шей жизни стрость
Epoch 153. Time: 0.567, Train loss: 0.364
бы нам соседе насе
Epoch 154. Time: 0.583, Train lo

Epoch 273. Time: 0.576, Train loss: 0.361
ать пртокинеть
Epoch 274. Time: 0.586, Train loss: 0.361
ёт и тамен порувства
Epoch 275. Time: 0.588, Train loss: 0.361
рыгки милудь и пол
Epoch 276. Time: 0.602, Train loss: 0.361
эть по другло не
Epoch 277. Time: 0.579, Train loss: 0.361
ять зубемь
Epoch 278. Time: 0.571, Train loss: 0.361
ять зубемь
Epoch 279. Time: 0.588, Train loss: 0.361
ых любил
Epoch 280. Time: 0.589, Train loss: 0.361
ёт и тамен порувства
Epoch 281. Time: 0.574, Train loss: 0.361
носкушей не ветрей меский даку 
Epoch 282. Time: 0.594, Train loss: 0.361
ых любил
Epoch 283. Time: 0.575, Train loss: 0.361
дов ро де
Epoch 284. Time: 0.577, Train loss: 0.361
 былог сладков востатья одна пр
Epoch 285. Time: 0.577, Train loss: 0.361
цей огдно опдало
Epoch 286. Time: 0.569, Train loss: 0.361
дов ро де
Epoch 287. Time: 0.581, Train loss: 0.361
ейдцений разнии в ним
Epoch 288. Time: 0.588, Train loss: 0.361
ой зажетей велил он
Epoch 289. Time: 0.567, Train loss: 0.361
зывоеться


Epoch 407. Time: 0.584, Train loss: 0.357
вожатн поравляливы злакой
Epoch 408. Time: 0.583, Train loss: 0.357
хоротый был ей
Epoch 409. Time: 0.580, Train loss: 0.357
зывоеться
Epoch 410. Time: 0.577, Train loss: 0.357
ков обребуеть 
Epoch 411. Time: 0.594, Train loss: 0.357
ейдцений разнии в ним
Epoch 412. Time: 0.600, Train loss: 0.357
рыгки милудь и пол
Epoch 413. Time: 0.599, Train loss: 0.357
тойник грустно в обусь пондобны
Epoch 414. Time: 0.542, Train loss: 0.357
йсёт ро стал 
Epoch 415. Time: 0.599, Train loss: 0.357
йсёт ро стал 
Epoch 416. Time: 0.582, Train loss: 0.357
ых любил
Epoch 417. Time: 0.578, Train loss: 0.357
стойн
Epoch 418. Time: 0.591, Train loss: 0.357
бы нам постаюб насудтей
Epoch 419. Time: 0.568, Train loss: 0.357
бы нам постаюб насудтей
Epoch 420. Time: 0.589, Train loss: 0.357
ков обребуеть 
Epoch 421. Time: 0.570, Train loss: 0.357
рыгки милудь и пол
Epoch 422. Time: 0.557, Train loss: 0.357
ьбо жино невеста праздат
Epoch 423. Time: 0.594, Train loss: 0.3

Epoch 545. Time: 0.580, Train loss: 0.354
ьбо жино невеста призлан
Epoch 546. Time: 0.610, Train loss: 0.354
зывоеться
Epoch 547. Time: 0.586, Train loss: 0.354
вожатн поравляливы лисаты
Epoch 548. Time: 0.583, Train loss: 0.354
дов ро де
Epoch 549. Time: 0.588, Train loss: 0.354
стойн
Epoch 550. Time: 0.598, Train loss: 0.354
дов ро де
Epoch 551. Time: 0.600, Train loss: 0.354
ять зубемь
Epoch 552. Time: 0.576, Train loss: 0.354
ой зажетей волвеном
Epoch 553. Time: 0.566, Train loss: 0.354
ать пртовинетьмы
Epoch 554. Time: 0.565, Train loss: 0.354
поэмукало к нетск
Epoch 555. Time: 0.606, Train loss: 0.354
ёт и тамен порувства
Epoch 556. Time: 0.590, Train loss: 0.354
утьсять нет
Epoch 557. Time: 0.618, Train loss: 0.354
эть по другло не
Epoch 558. Time: 0.643, Train loss: 0.354
ять зубемь
Epoch 559. Time: 0.579, Train loss: 0.354
хоротый был ей
Epoch 560. Time: 0.602, Train loss: 0.354
йсёт ро стал 
Epoch 561. Time: 0.597, Train loss: 0.354
дов ро де
Epoch 562. Time: 0.627, Train los

Epoch 682. Time: 0.593, Train loss: 0.352
го ясливын и зат
Epoch 683. Time: 0.629, Train loss: 0.352
фомо женний радство хлености
Epoch 684. Time: 0.594, Train loss: 0.351
женный проконе 
Epoch 685. Time: 0.603, Train loss: 0.351
none после ей наедине
Epoch 686. Time: 0.599, Train loss: 0.351
тойник грустно в обусь понразь
Epoch 687. Time: 0.593, Train loss: 0.351
бы нам посвят его
Epoch 688. Time: 0.600, Train loss: 0.351
ать пртовинетьмы
Epoch 689. Time: 0.576, Train loss: 0.351
none после ей наедине
Epoch 690. Time: 0.715, Train loss: 0.351
эть по другло на
Epoch 691. Time: 0.566, Train loss: 0.351
ьбо жино невеста призлан
Epoch 692. Time: 0.607, Train loss: 0.351
ять зубемь
Epoch 693. Time: 0.569, Train loss: 0.351
стойн
Epoch 694. Time: 0.575, Train loss: 0.351
ейдцений разнии в ним
Epoch 695. Time: 0.600, Train loss: 0.351
цей огдно опдало
Epoch 696. Time: 0.578, Train loss: 0.351
ейдцений разнии в ним
Epoch 697. Time: 0.649, Train loss: 0.351
ёт и тамен порувства
Epoch 698. Time

Epoch 818. Time: 0.597, Train loss: 0.348
ых любил
Epoch 819. Time: 0.572, Train loss: 0.656
ять зубемьяма
Epoch 820. Time: 0.771, Train loss: 0.617
тойник груст не учехот
Epoch 821. Time: 0.703, Train loss: 0.498
цей жет его семьес
Epoch 822. Time: 0.677, Train loss: 0.428
вожатн поравляливы лисаты
Epoch 823. Time: 0.609, Train loss: 0.390
лывы ни стеши
Epoch 824. Time: 0.607, Train loss: 0.373
юбый убо той
Epoch 825. Time: 0.609, Train loss: 0.364
ых любил
Epoch 826. Time: 0.581, Train loss: 0.360
зывоеться
Epoch 827. Time: 0.574, Train loss: 0.356
утьсять ять им мы сля вы
Epoch 828. Time: 0.552, Train loss: 0.353
хородных звуканал
Epoch 829. Time: 0.563, Train loss: 0.352
none после ей наедине
Epoch 830. Time: 0.593, Train loss: 0.351
ёт и тамен бынами
Epoch 831. Time: 0.566, Train loss: 0.350
ёт и тамен бынами
Epoch 832. Time: 0.559, Train loss: 0.350
none после ей наедине
Epoch 833. Time: 0.587, Train loss: 0.350
ков обребуеть 
Epoch 834. Time: 0.642, Train loss: 0.349
фомо женски

Epoch 954. Time: 0.546, Train loss: 0.344
 былог сладков отол
Epoch 955. Time: 0.595, Train loss: 0.344
поэмукало к нетск
Epoch 956. Time: 0.613, Train loss: 0.344
вожатн поравляливы лисаты
Epoch 957. Time: 0.605, Train loss: 0.344
чивыть
Epoch 958. Time: 0.590, Train loss: 0.344
лы был он сех 
Epoch 959. Time: 0.564, Train loss: 0.344
ёт и тамен порувства
Epoch 960. Time: 0.631, Train loss: 0.344
стопот 
Epoch 961. Time: 0.594, Train loss: 0.344
женной промучень много
Epoch 962. Time: 0.582, Train loss: 0.344
щей пон пеленый и ножато
Epoch 963. Time: 0.569, Train loss: 0.344
шей жиз мимы
Epoch 964. Time: 0.572, Train loss: 0.344
ёт и тамен порувства
Epoch 965. Time: 0.626, Train loss: 0.344
носкушей не веом
Epoch 966. Time: 0.590, Train loss: 0.344
мей пр волпечно увяюсь да
Epoch 967. Time: 0.558, Train loss: 0.344
ёт и тамен порувства
Epoch 968. Time: 0.575, Train loss: 0.344
щей пон пеленый и ножато
Epoch 969. Time: 0.572, Train loss: 0.344
 былог сладков отол
Epoch 970. Time: 0.567

In [17]:
class Network2(torch.nn.Module):

    def __init__(self):
        super(Network2, self).__init__()
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 35)
        self.gru = torch.nn.RNN(35, 256, 3, batch_first=True)
        self.hidden2tag = torch.nn.Linear(256, len(INDEX_TO_CHAR))

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 256))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 256))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [18]:
model2 = Network2()

In [19]:
criterion2 = torch.nn.CrossEntropyLoss()
optimizer2 = torch.optim.SGD(model2.parameters(), lr=.01)

In [20]:
for ep in range(10000):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer2.zero_grad()
        answers, _ = model2.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion2(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer2.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    generate_sentence(model2)


Epoch 0. Time: 1.418, Train loss: 3.500
ъ
Epoch 1. Time: 1.407, Train loss: 3.380
 
Epoch 2. Time: 1.462, Train loss: 3.226
б
Epoch 3. Time: 1.529, Train loss: 3.072
п
Epoch 4. Time: 1.461, Train loss: 2.990
с
Epoch 5. Time: 1.426, Train loss: 2.944
ю
Epoch 6. Time: 1.452, Train loss: 2.906
п
Epoch 7. Time: 1.491, Train loss: 2.873
у
Epoch 8. Time: 1.479, Train loss: 2.844
л
Epoch 9. Time: 1.481, Train loss: 2.819
ч
Epoch 10. Time: 1.499, Train loss: 2.796
т 
Epoch 11. Time: 1.518, Train loss: 2.776
р 
Epoch 12. Time: 1.459, Train loss: 2.758
 
Epoch 13. Time: 1.461, Train loss: 2.742
ё 
Epoch 14. Time: 1.462, Train loss: 2.728
бо
Epoch 15. Time: 1.443, Train loss: 2.715
п                              
Epoch 16. Time: 1.483, Train loss: 2.703
с
Epoch 17. Time: 1.396, Train loss: 2.692
о
Epoch 18. Time: 1.418, Train loss: 2.683
э
Epoch 19. Time: 1.383, Train loss: 2.674
де                             
Epoch 20. Time: 1.422, Train loss: 2.665
де                             
Epoch 21. Tim

Epoch 122. Time: 1.492, Train loss: 2.278
бала поло но нола но но но но н
Epoch 123. Time: 1.568, Train loss: 2.275
ы поло нола но но но но но но н
Epoch 124. Time: 1.696, Train loss: 2.273
ы поло нола но но но но но но н
Epoch 125. Time: 1.632, Train loss: 2.270
ёе поло нола но но но но но но 
Epoch 126. Time: 1.579, Train loss: 2.268
а поло нола но но но но но но н
Epoch 127. Time: 1.732, Train loss: 2.265
воло нола но нола но но но но н
Epoch 128. Time: 1.703, Train loss: 2.263
чола соло но нола но но но но н
Epoch 129. Time: 1.551, Train loss: 2.260
ясто нола но но но но но но но 
Epoch 130. Time: 1.589, Train loss: 2.258
эосто ноло но соло но но но но 
Epoch 131. Time: 1.659, Train loss: 2.256
е поло соло но но но но но но н
Epoch 132. Time: 1.817, Train loss: 2.254
бала поло но нола но но но но н
Epoch 133. Time: 2.568, Train loss: 2.251
на поло соло но но но но но но 
Epoch 134. Time: 1.746, Train loss: 2.249
та поло но соло но но но но но 
Epoch 135. Time: 1.523, Train loss: 2.

Epoch 252. Time: 1.851, Train loss: 2.120
зам серени постой
Epoch 253. Time: 1.784, Train loss: 2.119
чов поло полони 
Epoch 254. Time: 1.817, Train loss: 2.118
уто полони ной пола
Epoch 255. Time: 1.796, Train loss: 2.118
ам полони но пола
Epoch 256. Time: 1.806, Train loss: 2.117
ласто полони ной пола
Epoch 257. Time: 1.833, Train loss: 2.116
ый посто сто столь
Epoch 258. Time: 1.745, Train loss: 2.116
хосто полони ной пола
Epoch 259. Time: 1.765, Train loss: 2.115
ь посто сто сто наль
Epoch 260. Time: 1.816, Train loss: 2.114
ума пороволь
Epoch 261. Time: 1.816, Train loss: 2.114
бый посто сто столь
Epoch 262. Time: 1.862, Train loss: 2.113
жет полоно полони
Epoch 263. Time: 1.813, Train loss: 2.112
ума пороволь
Epoch 264. Time: 1.794, Train loss: 2.112
ласто полони не пола
Epoch 265. Time: 1.860, Train loss: 2.111
йоло полони не пола
Epoch 266. Time: 1.861, Train loss: 2.111
ой пороволь о пола
Epoch 267. Time: 1.853, Train loss: 2.110
чов полоно полони
Epoch 268. Time: 1.869, Train

Epoch 388. Time: 1.578, Train loss: 2.043
щузаль о продень
Epoch 389. Time: 1.746, Train loss: 2.050
дала пороволь
Epoch 390. Time: 1.621, Train loss: 2.047
щузаль о продень
Epoch 391. Time: 1.643, Train loss: 2.043
сто пороволь о прода
Epoch 392. Time: 1.534, Train loss: 2.046
полени не полани
Epoch 393. Time: 1.658, Train loss: 2.045
ь пороволь о провот
Epoch 394. Time: 1.728, Train loss: 2.042
воли не поровол
Epoch 395. Time: 1.739, Train loss: 2.043
це пороволь о прода
Epoch 396. Time: 1.694, Train loss: 2.044
фо пороволь о прода
Epoch 397. Time: 1.769, Train loss: 2.041
ъто пороволь
Epoch 398. Time: 1.792, Train loss: 2.041
юсто пороволь
Epoch 399. Time: 1.666, Train loss: 2.045
чала не пороволь
Epoch 400. Time: 1.607, Train loss: 2.040
мой пороволь о прода
Epoch 401. Time: 1.674, Train loss: 2.037
то пороволь о прода
Epoch 402. Time: 1.754, Train loss: 2.041
им сто пороволь оне
Epoch 403. Time: 1.759, Train loss: 2.045
фо пороволь о прода
Epoch 404. Time: 1.567, Train loss: 2.037

Epoch 522. Time: 1.923, Train loss: 1.992
йородна сторани вора
Epoch 523. Time: 1.884, Train loss: 1.993
ужда сто не поровот
Epoch 524. Time: 1.831, Train loss: 1.990
шоросто не пороволь
Epoch 525. Time: 1.824, Train loss: 1.989
ровот сто волиный стод
Epoch 526. Time: 1.810, Train loss: 1.989
ый породна стора
Epoch 527. Time: 1.766, Train loss: 1.991
эородна сторани вора
Epoch 528. Time: 1.804, Train loss: 1.990
ужда сто не поровот
Epoch 529. Time: 1.846, Train loss: 1.988
це пороволь он волен
Epoch 530. Time: 1.982, Train loss: 1.987
ой пороволь о провот
Epoch 531. Time: 1.887, Train loss: 1.988
йородна сторани вора
Epoch 532. Time: 1.824, Train loss: 1.996
сто пороволь он волен
Epoch 533. Time: 1.810, Train loss: 1.986
юров поредный стода
Epoch 534. Time: 1.845, Train loss: 1.985
бый пороволь о провот
Epoch 535. Time: 1.825, Train loss: 1.985
ласто не пороволь
Epoch 536. Time: 1.837, Train loss: 1.988
це пороволь о провот
Epoch 537. Time: 1.874, Train loss: 1.987
им сториный стора
Ep

...

Epoch 9341. Time: 1.808, Train loss: 0.266
ьшеспира свотезлол
Epoch 9342. Time: 1.750, Train loss: 0.266
двор
Epoch 9343. Time: 1.866, Train loss: 0.266
цем
Epoch 9344. Time: 1.778, Train loss: 0.266
этим обмынтай 
Epoch 9345. Time: 1.853, Train loss: 0.266
ой денье
Epoch 9346. Time: 1.862, Train loss: 0.266
ъкдя миной профушт на
Epoch 9347. Time: 1.870, Train loss: 0.266
ажд
Epoch 9348. Time: 1.762, Train loss: 0.266
ыхский револкный
Epoch 9349. Time: 1.839, Train loss: 0.266
ыхский револкный
Epoch 9350. Time: 1.800, Train loss: 0.266
будь бы одавал
Epoch 9351. Time: 1.832, Train loss: 0.266
нескромный
Epoch 9352. Time: 1.774, Train loss: 0.266
тому старика имеет
Epoch 9353. Time: 1.885, Train loss: 0.266
щей
Epoch 9354. Time: 1.827, Train loss: 0.266
ковыншу мна меи
Epoch 9355. Time: 1.767, Train loss: 0.266
мольнем лутье мость
Epoch 9356. Time: 1.782, Train loss: 0.266
седстве мовыд ожнят отам
Epoch 9357. Time: 1.819, Train loss: 0.266
седстве мовыд ожнят отам
Epoch 9358. Time: 1.92

Epoch 9485. Time: 1.750, Train loss: 0.265
ажд
Epoch 9486. Time: 1.789, Train loss: 0.265
ажд
Epoch 9487. Time: 1.844, Train loss: 0.265
цем
Epoch 9488. Time: 1.876, Train loss: 0.265
щей
Epoch 9489. Time: 1.843, Train loss: 0.265
седстве мовыды
Epoch 9490. Time: 1.891, Train loss: 0.265
постят
Epoch 9491. Time: 1.762, Train loss: 0.265
двор
Epoch 9492. Time: 1.924, Train loss: 0.265
ой денье
Epoch 9493. Time: 1.884, Train loss: 0.265
нескромный
Epoch 9494. Time: 1.931, Train loss: 0.265
готравил гутрый замет
Epoch 9495. Time: 1.932, Train loss: 0.265
ей
Epoch 9496. Time: 1.787, Train loss: 0.265
шуж иншалсяны скойиалать
Epoch 9497. Time: 1.857, Train loss: 0.265
юбрыйный
Epoch 9498. Time: 1.772, Train loss: 0.265
йро ветреных аетрадск
Epoch 9499. Time: 1.778, Train loss: 0.265
звужить
Epoch 9500. Time: 1.904, Train loss: 0.265
готравил гутрый замет
Epoch 9501. Time: 1.773, Train loss: 0.265
двор
Epoch 9502. Time: 1.835, Train loss: 0.265
щей
Epoch 9503. Time: 1.842, Train loss: 0.265


Epoch 9632. Time: 1.825, Train loss: 0.265
щей
Epoch 9633. Time: 1.759, Train loss: 0.265
ковын узлаву
Epoch 9634. Time: 1.876, Train loss: 0.265
ёт любивый 
Epoch 9635. Time: 1.937, Train loss: 0.265
юбрыйный
Epoch 9636. Time: 1.991, Train loss: 0.265
чень
Epoch 9637. Time: 1.886, Train loss: 0.265
седстве мовыды
Epoch 9638. Time: 1.890, Train loss: 0.265
ей
Epoch 9639. Time: 1.882, Train loss: 0.265
седстве мовыды
Epoch 9640. Time: 1.756, Train loss: 0.265
будь бы одавал
Epoch 9641. Time: 1.890, Train loss: 0.265
ёт любивый 
Epoch 9642. Time: 1.832, Train loss: 0.265
нескромный
Epoch 9643. Time: 1.981, Train loss: 0.265
постят
Epoch 9644. Time: 1.890, Train loss: 0.265
этим обмынтий так л
Epoch 9645. Time: 1.853, Train loss: 0.265
имссо красил
Epoch 9646. Time: 1.852, Train loss: 0.265
юбрыйный
Epoch 9647. Time: 1.872, Train loss: 0.265
юбрыйный
Epoch 9648. Time: 1.919, Train loss: 0.265
звужить
Epoch 9649. Time: 1.838, Train loss: 0.265
звужить
Epoch 9650. Time: 1.780, Train loss: 0

Epoch 9778. Time: 1.816, Train loss: 0.264
рожалсойд
Epoch 9779. Time: 1.775, Train loss: 0.264
юбрыйный
Epoch 9780. Time: 1.869, Train loss: 0.264
ажд
Epoch 9781. Time: 1.944, Train loss: 0.264
чень
Epoch 9782. Time: 1.963, Train loss: 0.264
имссо красил
Epoch 9783. Time: 1.832, Train loss: 0.264
двор
Epoch 9784. Time: 1.799, Train loss: 0.264
ёт любивый 
Epoch 9785. Time: 1.801, Train loss: 0.264
ьшеспира свотезлол
Epoch 9786. Time: 1.950, Train loss: 0.264
цем
Epoch 9787. Time: 1.825, Train loss: 0.264
ажд
Epoch 9788. Time: 1.762, Train loss: 0.264
уж вля
Epoch 9789. Time: 1.791, Train loss: 0.264
ей
Epoch 9790. Time: 1.853, Train loss: 0.264
рожалсойд
Epoch 9791. Time: 1.795, Train loss: 0.264
юбрыйный
Epoch 9792. Time: 1.855, Train loss: 0.264
седстве мовыды
Epoch 9793. Time: 1.866, Train loss: 0.264
ъкдя миной профушт на
Epoch 9794. Time: 1.875, Train loss: 0.264
нескромный
Epoch 9795. Time: 1.827, Train loss: 0.264
 взумовеошла давежен
Epoch 9796. Time: 1.840, Train loss: 0.264


Epoch 9924. Time: 1.797, Train loss: 0.264
шуж иншалсяны скойиалать
Epoch 9925. Time: 1.910, Train loss: 0.264
хотяск
Epoch 9926. Time: 1.863, Train loss: 0.264
юбрыйный
Epoch 9927. Time: 1.926, Train loss: 0.264
щей
Epoch 9928. Time: 1.875, Train loss: 0.264
водшляд
Epoch 9929. Time: 1.883, Train loss: 0.264
йро ветреных аетрадск
Epoch 9930. Time: 1.783, Train loss: 0.264
ковын ездых вол не сврегиво
Epoch 9931. Time: 1.817, Train loss: 0.264
ёт любивый 
Epoch 9932. Time: 1.927, Train loss: 0.264
ёт любивый 
Epoch 9933. Time: 1.802, Train loss: 0.264
звужить
Epoch 9934. Time: 1.846, Train loss: 0.264
ажд
Epoch 9935. Time: 1.848, Train loss: 0.264
ёт любивый 
Epoch 9936. Time: 1.854, Train loss: 0.264
щей
Epoch 9937. Time: 1.820, Train loss: 0.264
ять людий
Epoch 9938. Time: 1.758, Train loss: 0.264
будь бы одавал
Epoch 9939. Time: 1.756, Train loss: 0.264
ыхский револкный
Epoch 9940. Time: 1.802, Train loss: 0.264
ъкдя миной профушт на
Epoch 9941. Time: 1.825, Train loss: 0.264
ёт люби

In [21]:
class Network3(torch.nn.Module):

    def __init__(self):
        super(Network3, self).__init__()
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 35)
        self.gru = torch.nn.RNN(35, 512,3, batch_first=True)
        self.hidden2tag = torch.nn.Linear(512, len(INDEX_TO_CHAR))

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 512))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 512))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [22]:
model3 = Network3()

In [23]:
criterion3 = torch.nn.CrossEntropyLoss()
optimizer3 = torch.optim.SGD(model3.parameters(), lr=.01)

In [24]:
for ep in range(10000):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer3.zero_grad()
        answers, _ = model3.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion3(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer3.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    generate_sentence(model3)


Epoch 0. Time: 1.385, Train loss: 3.497
яи 
Epoch 1. Time: 1.229, Train loss: 3.350
и
Epoch 2. Time: 1.003, Train loss: 3.139
т
Epoch 3. Time: 1.052, Train loss: 2.970
ъ
Epoch 4. Time: 1.272, Train loss: 2.906
к
Epoch 5. Time: 1.046, Train loss: 2.862
я
Epoch 6. Time: 1.050, Train loss: 2.824
я 
Epoch 7. Time: 1.038, Train loss: 2.793
ю
Epoch 8. Time: 1.022, Train loss: 2.767
ы
Epoch 9. Time: 1.034, Train loss: 2.746
х
Epoch 10. Time: 1.075, Train loss: 2.728
ли                             
Epoch 11. Time: 1.006, Train loss: 2.713
п                              
Epoch 12. Time: 1.152, Train loss: 2.700
ч  о                           
Epoch 13. Time: 1.089, Train loss: 2.689
и  о  о     о     о  о     о   
Epoch 14. Time: 1.104, Train loss: 2.679
ли  о  о  о  о  о  о  о  о  о  
Epoch 15. Time: 1.248, Train loss: 2.670
ф  о  о  о  о  о  о  о  о  о  о
Epoch 16. Time: 1.421, Train loss: 2.662
у  о  о  о  о  о  о  о  о  о  о
Epoch 17. Time: 1.207, Train loss: 2.654
я ле  о  о  о  о  о  о  о

Epoch 119. Time: 1.188, Train loss: 2.255
ка пола на ной полина
Epoch 120. Time: 1.125, Train loss: 2.253
эи пола на но нали на ной
Epoch 121. Time: 1.115, Train loss: 2.250
ф пола на ной полина
Epoch 122. Time: 1.129, Train loss: 2.248
у пола на сола на ной
Epoch 123. Time: 1.123, Train loss: 2.246
го пола на сола на ной
Epoch 124. Time: 1.163, Train loss: 2.244
цо пола на сола на ной
Epoch 125. Time: 1.375, Train loss: 2.242
о пола на сола на ной
Epoch 126. Time: 1.138, Train loss: 2.239
ь пола на сола на ной 
Epoch 127. Time: 1.121, Train loss: 2.237
е пола на сола на ной
Epoch 128. Time: 1.184, Train loss: 2.235
зала пола на ной поли
Epoch 129. Time: 1.490, Train loss: 2.233
бо пола на сола на ной
Epoch 130. Time: 1.308, Train loss: 2.231
а пола на сола на ной
Epoch 131. Time: 1.199, Train loss: 2.229
дал со пола на ной поли
Epoch 132. Time: 1.237, Train loss: 2.227
ы пола на сола на ной
Epoch 133. Time: 1.212, Train loss: 2.225
дал со пола на ной поли
Epoch 134. Time: 1.168, Train

Epoch 252. Time: 1.291, Train loss: 2.106
ы пораль о продой
Epoch 253. Time: 1.166, Train loss: 2.106
мой пораль о продой
Epoch 254. Time: 1.133, Train loss: 2.105
бой пораль о продой
Epoch 255. Time: 1.208, Train loss: 2.104
ца пораль о продой
Epoch 256. Time: 1.471, Train loss: 2.104
я пораль о продой
Epoch 257. Time: 1.281, Train loss: 2.103
сто пораль о продой
Epoch 258. Time: 1.135, Train loss: 2.103
ный пораль о продой
Epoch 259. Time: 1.108, Train loss: 2.102
заль о продо на столь
Epoch 260. Time: 1.161, Train loss: 2.101
ой пораль о продой
Epoch 261. Time: 1.206, Train loss: 2.101
х пораль о продоль
Epoch 262. Time: 1.252, Train loss: 2.100
ёто пораль о прода
Epoch 263. Time: 1.164, Train loss: 2.100
й пораль о продоль
Epoch 264. Time: 1.231, Train loss: 2.099
мой пораль о продой
Epoch 265. Time: 1.140, Train loss: 2.098
и пораль о продой
Epoch 266. Time: 1.271, Train loss: 2.098
ой пораль о продой
Epoch 267. Time: 1.153, Train loss: 2.097
я пораль о продой
Epoch 268. Time: 1.1

Epoch 384. Time: 1.154, Train loss: 2.039
ой порали не стора
Epoch 385. Time: 1.120, Train loss: 2.036
разной порали на стора
Epoch 386. Time: 1.121, Train loss: 2.038
вой порали не стора
Epoch 387. Time: 1.256, Train loss: 2.035
то порали не стора
Epoch 388. Time: 1.129, Train loss: 2.037
кой порали не стора
Epoch 389. Time: 1.184, Train loss: 2.035
ца порали не стора
Epoch 390. Time: 1.125, Train loss: 2.035
ный порали на стора
Epoch 391. Time: 1.151, Train loss: 2.035
ши порали на стора
Epoch 392. Time: 1.332, Train loss: 2.033
разной порали на стора
Epoch 393. Time: 1.180, Train loss: 2.036
чаль о провол се дала
Epoch 394. Time: 1.114, Train loss: 2.030
еть о провол се дала
Epoch 395. Time: 1.195, Train loss: 2.042
заль о проволь о прода
Epoch 396. Time: 1.150, Train loss: 2.030
дой порали не стора
Epoch 397. Time: 1.150, Train loss: 2.034
ъоради пораль о прода
Epoch 398. Time: 1.120, Train loss: 2.032
ужь о провол сера
Epoch 399. Time: 1.122, Train loss: 2.033
ь порали не стора
Ep

Epoch 516. Time: 1.225, Train loss: 1.984
вой порадил се дала
Epoch 517. Time: 1.207, Train loss: 1.986
го простали не стора
Epoch 518. Time: 1.242, Train loss: 1.982
стой порадил се дала
Epoch 519. Time: 1.146, Train loss: 1.981
ить о простали не стора
Epoch 520. Time: 1.328, Train loss: 1.989
 порадал се дала на стора
Epoch 521. Time: 1.209, Train loss: 1.981
ъторани провот о пора
Epoch 522. Time: 1.127, Train loss: 1.982
хо простали не стора
Epoch 523. Time: 1.127, Train loss: 1.983
той порадил се дала
Epoch 524. Time: 1.300, Train loss: 1.981
ластой порадил се дала
Epoch 525. Time: 1.121, Train loss: 1.981
ь породна сторани 
Epoch 526. Time: 1.151, Train loss: 1.981
ца породна сторани
Epoch 527. Time: 1.205, Train loss: 1.979
ь породна сторани 
Epoch 528. Time: 1.152, Train loss: 1.980
юто на поралань о прода
Epoch 529. Time: 1.119, Train loss: 1.980
ить о простали не стора
Epoch 530. Time: 1.143, Train loss: 1.978
ый порадил се дала
Epoch 531. Time: 1.166, Train loss: 1.979
аль о 

...

Epoch 9213. Time: 1.156, Train loss: 0.258
ьмнинецум и вется
Epoch 9214. Time: 1.190, Train loss: 0.258
 встой тамит он сведанья
Epoch 9215. Time: 1.191, Train loss: 0.258
 встой тамит он сведанья
Epoch 9216. Time: 1.154, Train loss: 0.258
форт
Epoch 9217. Time: 1.392, Train loss: 0.258
нескромный
Epoch 9218. Time: 1.250, Train loss: 0.258
му ножки иль брадлиж 
Epoch 9219. Time: 1.260, Train loss: 0.258
желня
Epoch 9220. Time: 1.171, Train loss: 0.258
коне
Epoch 9221. Time: 1.312, Train loss: 0.258
людаю
Epoch 9222. Time: 1.200, Train loss: 0.258
му ножки иль брадлиж 
Epoch 9223. Time: 1.297, Train loss: 0.258
эти мбугом
Epoch 9224. Time: 1.489, Train loss: 0.258
желня
Epoch 9225. Time: 1.139, Train loss: 0.258
ревстваршла
Epoch 9226. Time: 1.312, Train loss: 0.258
ться
Epoch 9227. Time: 1.209, Train loss: 0.258
му ножки иль брадлиж 
Epoch 9228. Time: 1.106, Train loss: 0.258
ться
Epoch 9229. Time: 1.220, Train loss: 0.258
чув
Epoch 9230. Time: 1.182, Train loss: 0.258
людаю
Epoch 9231

Epoch 9370. Time: 1.293, Train loss: 0.258
двор
Epoch 9371. Time: 1.320, Train loss: 0.258
 встой тамин возручен
Epoch 9372. Time: 1.392, Train loss: 0.258
желня
Epoch 9373. Time: 1.191, Train loss: 0.258
отруж
Epoch 9374. Time: 1.126, Train loss: 0.258
отруж
Epoch 9375. Time: 1.119, Train loss: 0.258
ам сумдай
Epoch 9376. Time: 1.315, Train loss: 0.258
ётсы
Epoch 9377. Time: 1.122, Train loss: 0.258
 встой тамин возручен
Epoch 9378. Time: 1.259, Train loss: 0.258
му ножки иль брадлиж 
Epoch 9379. Time: 1.147, Train loss: 0.258
готов отрый
Epoch 9380. Time: 1.116, Train loss: 0.258
отруж
Epoch 9381. Time: 1.234, Train loss: 0.258
желня
Epoch 9382. Time: 1.173, Train loss: 0.258
ам сумдай
Epoch 9383. Time: 1.122, Train loss: 0.258
эти мбугом
Epoch 9384. Time: 1.115, Train loss: 0.258
звитель мои лиря
Epoch 9385. Time: 1.213, Train loss: 0.258
му ножки иль брадлиж 
Epoch 9386. Time: 1.174, Train loss: 0.258
нескромный
Epoch 9387. Time: 1.135, Train loss: 0.258
ам сумдай
Epoch 9388. Time:

Epoch 9527. Time: 1.329, Train loss: 0.258
коне
Epoch 9528. Time: 1.511, Train loss: 0.258
коне
Epoch 9529. Time: 1.247, Train loss: 0.258
ить до
Epoch 9530. Time: 1.276, Train loss: 0.258
ють
Epoch 9531. Time: 1.218, Train loss: 0.258
двор
Epoch 9532. Time: 1.192, Train loss: 0.258
 встой тамин возручен
Epoch 9533. Time: 1.256, Train loss: 0.258
шусь
Epoch 9534. Time: 1.163, Train loss: 0.258
ых 
Epoch 9535. Time: 1.113, Train loss: 0.258
форт
Epoch 9536. Time: 1.210, Train loss: 0.258
ться
Epoch 9537. Time: 1.140, Train loss: 0.258
шусь
Epoch 9538. Time: 1.133, Train loss: 0.258
ться
Epoch 9539. Time: 1.114, Train loss: 0.258
звитель мои
Epoch 9540. Time: 1.146, Train loss: 0.258
людаю
Epoch 9541. Time: 1.256, Train loss: 0.258
статей
Epoch 9542. Time: 1.485, Train loss: 0.258
 встой тамин возручен
Epoch 9543. Time: 1.322, Train loss: 0.258
ых 
Epoch 9544. Time: 1.135, Train loss: 0.258
шусь
Epoch 9545. Time: 1.110, Train loss: 0.258
ить до
Epoch 9546. Time: 1.120, Train loss: 0.258


Epoch 9687. Time: 1.155, Train loss: 0.258
чув
Epoch 9688. Time: 1.139, Train loss: 0.258
бовь
Epoch 9689. Time: 1.121, Train loss: 0.258
ревстваршла
Epoch 9690. Time: 1.197, Train loss: 0.258
 встой тамин возручен
Epoch 9691. Time: 1.143, Train loss: 0.258
эти мбугом
Epoch 9692. Time: 1.335, Train loss: 0.258
желня
Epoch 9693. Time: 1.158, Train loss: 0.258
ться
Epoch 9694. Time: 1.243, Train loss: 0.258
ють
Epoch 9695. Time: 1.397, Train loss: 0.258
ам сумдай
Epoch 9696. Time: 1.242, Train loss: 0.258
чув
Epoch 9697. Time: 1.180, Train loss: 0.258
двор
Epoch 9698. Time: 1.213, Train loss: 0.258
двор
Epoch 9699. Time: 1.149, Train loss: 0.258
двор
Epoch 9700. Time: 1.119, Train loss: 0.258
шусь
Epoch 9701. Time: 1.163, Train loss: 0.258
ам сумдай
Epoch 9702. Time: 1.115, Train loss: 0.258
етст
Epoch 9703. Time: 1.132, Train loss: 0.258
чув
Epoch 9704. Time: 1.219, Train loss: 0.258
ьмнинецум и вется
Epoch 9705. Time: 1.184, Train loss: 0.258
людаю
Epoch 9706. Time: 1.117, Train loss: 

Epoch 9847. Time: 1.153, Train loss: 0.258
етст
Epoch 9848. Time: 1.177, Train loss: 0.258
ревстваршла
Epoch 9849. Time: 1.488, Train loss: 0.258
ых 
Epoch 9850. Time: 1.260, Train loss: 0.258
людаю
Epoch 9851. Time: 1.188, Train loss: 0.258
ють
Epoch 9852. Time: 1.188, Train loss: 0.258
ам сумдай
Epoch 9853. Time: 1.139, Train loss: 0.258
ють
Epoch 9854. Time: 1.124, Train loss: 0.258
желня
Epoch 9855. Time: 1.200, Train loss: 0.258
ых 
Epoch 9856. Time: 1.132, Train loss: 0.258
ъевнявью 
Epoch 9857. Time: 1.212, Train loss: 0.258
му ножки иль брадлиж 
Epoch 9858. Time: 1.119, Train loss: 0.258
ьмнинецум и вется
Epoch 9859. Time: 1.127, Train loss: 0.258
коне
Epoch 9860. Time: 1.285, Train loss: 0.258
статей
Epoch 9861. Time: 1.198, Train loss: 0.258
цела
Epoch 9862. Time: 1.327, Train loss: 0.258
нескромный
Epoch 9863. Time: 1.257, Train loss: 0.258
готов отрый
Epoch 9864. Time: 1.219, Train loss: 0.258
ьмнинецум и вется
Epoch 9865. Time: 1.151, Train loss: 0.258
му ножки иль брадлиж

In [ ]:
# Лучше всего сработала модель с размером эммбединга 35, и размером скрытого слоя 256